# ECG Classification

## Project Goal
The goal of this project is to create a deep neural network able to classify ECG signals and cardiac arrhythmia with a high degree of accuracy. Alongside this goal is the aim to explore the implications of introducing simulated data into the training sets in hopes of improving the overall accuracy of the model.

The model created and used for this project is based off of the model preseneted in the Yildrim20 paper :
https://www.sciencedirect.com/science/article/abs/pii/S016926072031573X


### Firstly import all of the neccesary libraries and mount google drive to load data


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import scipy.io

from keras.models import Sequential, load_model
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization, LeakyReLU, Dropout, LSTM, Flatten, Dense 
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelBinarizer
from matplotlib import pyplot

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Function to load the relevant data
Ammend the file path based upon where data is stored

In [ ]:
def load_data():

  # Import the data
  data = pd.read_csv("/content/drive/My Drive/ECG/ammendedData/mergedData.csv").values
  labels = pd.read_csv("/content/drive/My Drive/ECG/ammendedData/mergedLabels.csv").values

  # Converts labels to a binary format
  encoder = LabelBinarizer()
  binaryLabels = encoder.fit_transform(labels)

  return data, binaryLabels

### Function to create the model


In [ ]:
def create_model():
  # Initialise model
  model = Sequential()

  # 'Input' layer chunk
  model.add( Conv1D( filters=64, kernel_size=21, strides=11, input_shape=[5000, 1] ) )
  model.add( MaxPooling1D( pool_size=2 ) )
  model.add( BatchNormalization() )
  model.add( LeakyReLU( alpha=0.1 ) )
  model.add( Dropout( rate=0.3 ) )

  # Hidden layers
  model.add( Conv1D( filters=64, kernel_size=7, strides=1, input_shape=[64, 226] ) )
  model.add( MaxPooling1D( pool_size=2 ) )
  model.add( BatchNormalization() )

  model.add( Conv1D( filters=128, kernel_size=5, strides=1, input_shape=[64, 110] ) )
  model.add( MaxPooling1D( pool_size=2 ) )

  model.add( Conv1D( filters=256, kernel_size=13, strides=1, input_shape=[128, 53] ) )
  model.add( Conv1D( filters=512, kernel_size=7, strides=1, input_shape=[256, 41] ) )
  model.add( Dropout( rate=0.3 ) )

  model.add( Conv1D( filters=256, kernel_size=9, strides=1, input_shape=[512, 35] ) )
  model.add( MaxPooling1D( pool_size=2 ) )

  # 'Output' layer chunk
  model.add( LSTM( units=128, return_sequences=True) )
  model.add( Flatten() )
  model.add( Dense( units=64, activation='relu' ) )
  model.add( Dense( units=1, activation='sigmoid' ) )

  # Create optimizer with custom learning rate
  opt = tf.keras.optimizers.Adam(learning_rate=0.0002)

  # Compile the model
  model.compile( loss="binary_crossentropy", optimizer=opt, metrics=['binary_accuracy', tf.keras.metrics.TruePositives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalseNegatives()] )

  return model


### Train the model
Ammend the file path in model checkpoint in order too save model in desired file

In [ ]:
def train_model(model, X_train, Y_train, X_test, Y_test):
  X_train = X_train[..., None]
  X_test = X_test[..., None]
  #X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
  #X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)

  earlyStopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  modelCheckpoint = ModelCheckpoint('/content/drive/My Drive/ECG/real75sim25Model/model-{epoch:03d}-{val_loss:03f}.hdf5', monitor='val_loss')
  reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001 )

  history = model.fit( X_train, Y_train,  epochs=60, validation_data=(X_test,Y_test), callbacks=[earlyStopping, modelCheckpoint, reduceLR] )
  
  return history

In [ ]:
def getLabelIndex( train_index, data, labels ):

  # initialise array
  normal_index = []
  atrial_index = []

  # for each element only keep desired  N=1  A=0
  for i in range(train_index.size):
    if labels[i] == 1:
      normal_index.append(train_index[i])
    else:
      atrial_index.append(train_index[i])

  # convert to numpy array
  normal_index = np.array(normal_index)
  atrial_index = np.array(atrial_index)

  return normal_index, atrial_index

### Create Kfolds and train the CNN



In [ ]:
total_folds=5
i=1

# Load the data
data, labels = load_data()

# Note all simulated data are appeneded to the end of file
# No way to check as all stored in identical way to real data so needs hardcoded
total_simulated = 622

# Total size of entire dataset
total_elements = int(data.size / 5000) # 5000 becuase length of each individual record

# Total real data in dataset
total_real = total_elements - total_simulated

# Create index for simulated data
if (total_simulated>0):
  simulated_index = np.arange(total_real, total_elements)

# Create K folds ensuring theyre balanced
skf = StratifiedKFold(total_folds,  True)

# Train CNN
# Select only the real data to make splits to ensure test split is only real data
for train_index, test_index in skf.split( data[:total_real, :], labels[:total_real, :] ):

  # Use to get classes index    #### Comment out next 2 lines if not needed for model ####
  normal_index, atrial_index = getLabelIndex(train_index, data, labels)
  train_index = normal_index

  # Add in the simulated datas index back to the training set so its used for training
  train_index = np.concatenate( (train_index, atrial_index[:373]) ) #### Adjust index of atrial_index to control amount of atrial data comment out line if 0 ####
  train_index = np.concatenate( (train_index, simulated_index[:125]) ) #### Adjust index of simulated_index to control amount of simulated data comment out line if 0 ####

  print("Running Fold", i, "/", total_folds)
  print('-----------------------------------------------------')

  model = create_model()

  # Create the test and training sets with relevant labes
  X_train, X_test = data[train_index], data[test_index]
  Y_train, Y_test = labels[train_index], labels[test_index]

  # Train the model
  history = train_model(model, X_train, Y_train, X_test, Y_test)

  # Save history
  filename = "/content/drive/My Drive/ECG/history/real75sim25_history_" + str(i) + ".csv"
  pd.DataFrame.from_dict(history.history).to_csv(filename, index=False)

  i=i+1

Running Fold 1 / 5
-----------------------------------------------------
Epoch 1/60
138/138 [==============================] - 80s 547ms/step - loss: 0.4263 - binary_accuracy: 0.8583 - true_positives_6: 1916.3165 - false_positives_6: 320.8921 - true_negatives_6: 0.5036 - false_negatives_6: 1.8849 - val_loss: 0.3807 - val_binary_accuracy: 0.8757 - val_true_positives_6: 972.0000 - val_false_positives_6: 138.0000 - val_true_negatives_6: 0.0000e+00 - val_false_negatives_6: 0.0000e+00
Epoch 2/60
138/138 [==============================] - 73s 526ms/step - loss: 0.3898 - binary_accuracy: 0.8580 - true_positives_6: 1876.1007 - false_positives_6: 288.0000 - true_negatives_6: 55.5612 - false_negatives_6: 19.9353 - val_loss: 0.3006 - val_binary_accuracy: 0.8946 - val_true_positives_6: 957.0000 - val_false_positives_6: 102.0000 - val_true_negatives_6: 36.0000 - val_false_negatives_6: 15.0000
Epoch 3/60
138/138 [==============================] - 73s 526ms/step - loss: 0.3189 - binary_accuracy: 0.89

In [ ]:
print("Average test loss: ", np.average(history.history['val_loss']))
